# Расширенное тестирование модуля формирования портфеля

Этот ноутбук демонстрирует расширенный цикл работы с модулем tvr_service.pipeline.portfolio — от загрузки данных по инструментам до расчёта распределения капитала с использованием индикаторного инструмента и параметров.


## 1. Подготовка окружения

Импортируем необходимые библиотеки и подключим модуль портфеля. Добавляем каталог src в sys.path, чтобы можно было использовать код проекта без установки пакета.


In [1]:
import sys
from pathlib import Path

#========================
# import pandas as pd
# import numpy as np

# # Настройка отображения
# pd.set_option('display.max_rows', None)        # Показывать все строки
# pd.set_option('display.max_columns', None)     # Показывать все колонки  
# pd.set_option('display.width', None)           # Не ограничивать ширину
# pd.set_option('display.max_colwidth', 50)      # Ограничить ширину колонок для читаемости
# pd.set_option('display.expand_frame_repr', False)  # Не переносить DataFrame на новую строку
#========================

PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

from tvr_service.pipeline import (
    allocations_to_frame,
    build_portfolio,
    filter_by_suffix,
    filter_by_whitelist,
    load_securities,
    load_whitelist,
    )

import pandas as pd


## 2. Настройка параметров теста

Задаём базовые параметры: объём капитала, необходимость скачивания свежих данных с MOEX, путь к локальному CSV (если он есть), фильтрацию по суффиксу тикера, количество параметров и индикаторный инструмент.


In [25]:
CAPITAL = 3_00_000  # общий капитал, руб.
PREFER_REMOTE = True   # при True сначала пробуем скачать данные с MOEX
SEC_DATA_FILE = None   # можно указать путь к локальному sec_tvr.csv
SUFFIX_FILTER = 'Z5'   # например, 'H5', 'M5' и т.п. для фьючерсов
NUM_PARAMS = 1         # количество параметров для каждого инструмента в портфеле
SEC_IND = 'MXZ5'       # код индикаторного инструмента (пример)

# Параметры для расчета float значений
float_1_x = 0.01  # коэффициент для float_1
float_2_x = 0.02   # коэффициент для float_2
float_3_x = 0.01   # значение для float_3
float_4_x = 0.05   # значение для float_4


## 3. Загрузка справочника инструментов

Попробуем получить таблицу инструментов. Функция load_securities автоматически рассчитает поле full_price и добавит тикеры TI* (клонов TB*). В процессе будет использован список из интернета или локального файла.


In [3]:
securities = load_securities(sec_data_file=SEC_DATA_FILE, prefer_remote=PREFER_REMOTE)
print(f"Загружено инструментов: {len(securities)}")
# securities.head()


Загружено инструментов: 400


## 4. Получение цены индикаторного инструмента

Найдем цену индикаторного инструмента в справочнике и сохраним её в переменную SEC_IND_PRICE.


In [4]:
# Поиск индикаторного инструмента в справочнике
indicator_security = securities[securities['SECID'] == SEC_IND]

if not indicator_security.empty:
    SEC_IND_PRICE = indicator_security['full_price'].iloc[0]
    print(f"Индикаторный инструмент {SEC_IND} найден. Цена: {SEC_IND_PRICE:,.2f} руб.")
    print(f"Краткое название: {indicator_security['SHORTNAME'].iloc[0]}")
else:
    print(f"Индикаторный инструмент {SEC_IND} не найден в справочнике!")
    print("Доступные инструменты с похожими кодами:")
    similar_securities = securities[securities['SECID'].str.contains(SEC_IND[:3], case=False)]
    if not similar_securities.empty:
        print(similar_securities[['SECID', 'SHORTNAME']].head())
    else:
        print("Похожих инструментов не найдено.")
    SEC_IND_PRICE = None


Индикаторный инструмент MXZ5 найден. Цена: 281,500.00 руб.
Краткое название: MIX-12.25


## 5. Фильтрация справочника

Сначала ограничиваем список инструментов по выбранному `SUFFIX_FILTER`, затем применяем whitelist. Так можно сузить вселенную до нужной серии и далее оставить только интересующие базовые активы.


In [5]:
try:
    whitelist = load_whitelist()
    print(f'Инструментов в whitelist: {len(whitelist)}')
except FileNotFoundError:
    whitelist = None
    print('Файл whitelist не найден. Будем использовать полную вселенную инструментов.')

# Фильтрация по суффиксу
base_selection = filter_by_suffix(securities, SUFFIX_FILTER)
print(f'Инструментов после фильтра по суффиксу {SUFFIX_FILTER}: {len(base_selection)}')

# Фильтрация по whitelist
if whitelist and not base_selection.empty:
    filtered_securities = filter_by_whitelist(base_selection, whitelist)
    print(f'Инструментов после применения whitelist: {len(filtered_securities)}')
elif whitelist:
    filtered_securities = pd.DataFrame(columns=base_selection.columns if not base_selection.empty else securities.columns)
    print('После фильтра по суффиксу подходящих инструментов нет.')
else:
    filtered_securities = base_selection

if not filtered_securities.empty:
    print('Примеры отфильтрованных инструментов:')
    display(filtered_securities[['SECID', 'SHORTNAME', 'base_code', 'full_price']].head())
    
    # Проверка уникальности по base_code
    print(f"\nПроверка уникальности по base_code:")
    unique_base_codes = filtered_securities['base_code'].nunique()
    total_instruments = len(filtered_securities)
    print(f"Уникальных base_code: {unique_base_codes}")
    print(f"Общее количество инструментов: {total_instruments}")
    
    if unique_base_codes != total_instruments:
        print("⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!")
        duplicates = filtered_securities.groupby('base_code').size()
        duplicates = duplicates[duplicates > 1]
        print(f"Дублирующиеся base_code ({len(duplicates)}):")
        for base_code, count in duplicates.head(10).items():
            print(f"  {base_code}: {count} инструментов")
            # Показываем примеры дубликатов
            examples = filtered_securities[filtered_securities['base_code'] == base_code][['SECID', 'SHORTNAME', 'base_code']]
            print(f"    Примеры: {examples['SECID'].tolist()}")
    else:
        print("✅ Все base_code уникальны")
        
else:
    print('После фильтрации инструментов не найдено.')


Инструментов в whitelist: 44
Инструментов после фильтра по суффиксу Z5: 131
Инструментов после применения whitelist: 45
Примеры отфильтрованных инструментов:


,SECID,SHORTNAME,base_code,full_price
0,AFZ5,AFLT-12.25,AFLT,5793.0
1,AKZ5,AFKS-12.25,AFKS,14687.0
2,ALZ5,ALRS-12.25,ALRS,4627.0
3,BNZ5,BANE-12.25,BANE,1600.0
4,CMZ5,CBOM-12.25,CBOM,7539.0



Проверка уникальности по base_code:
Уникальных base_code: 44
Общее количество инструментов: 45
⚠️  ВНИМАНИЕ: Найдены дубликаты по base_code!
Дублирующиеся base_code (1):
  T: 2 инструментов
    Примеры: ['TBZ5', 'TIZ5']


## 5.2. Альтернативный диалог (без ipywidgets)

Если ipywidgets недоступны, используем простой текстовый интерфейс для выбора дубликатов.

In [6]:
def remove_duplicates_simple(df, base_code_column='base_code'):
    """
    Простая функция для удаления дублирующихся инструментов по base_code.
    Использует текстовый интерфейс вместо ipywidgets.
    
    Args:
        df: DataFrame с инструментами
        base_code_column: название колонки для проверки дубликатов
    
    Returns:
        DataFrame без дубликатов
    """
    import pandas as pd
    
    # Находим дубликаты
    duplicates = df.groupby(base_code_column).size()
    duplicates = duplicates[duplicates > 1]
    
    if len(duplicates) == 0:
        print("✅ Дубликатов не найдено!")
        return df
    
    print(f"🔍 Найдено {len(duplicates)} групп дубликатов:")
    
    indices_to_remove = []
    
    for base_code in duplicates.index:
        group_data = df[df[base_code_column] == base_code]
        print(f"\n📋 Группа '{base_code}' ({len(group_data)} инструментов):")
        
        # Показываем инструменты в группе
        for i, (idx, row) in enumerate(group_data.iterrows()):
            print(f"  {i+1}. {row['SECID']} - {row['SHORTNAME']} (цена: {row['full_price']})")
        
        # Запрашиваем выбор
        print(f"\n❓ Какие инструменты из группы '{base_code}' удалить?")
        print("   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:")
        
        try:
            choice = input(2).strip()
            
            if choice.lower() == 'all':
                # Удаляем все кроме первого
                indices_to_remove.extend(group_data.index[1:].tolist())
                print(f"   ✅ Будет удалено {len(group_data)-1} инструментов (останется первый)")
            elif choice:
                # Парсим номера
                numbers = [int(x.strip()) for x in choice.split(',') if x.strip().isdigit()]
                if numbers:
                    # Проверяем валидность номеров
                    valid_numbers = [n for n in numbers if 1 <= n <= len(group_data)]
                    if valid_numbers:
                        # Получаем индексы для удаления
                        indices_to_remove.extend([group_data.index[n-1] for n in valid_numbers])
                        print(f"   ✅ Будет удалено {len(valid_numbers)} инструментов")
                    else:
                        print("   ⚠️ Некорректные номера, пропускаем группу")
                else:
                    print("   ⚠️ Некорректный ввод, пропускаем группу")
            else:
                print("   ⏭️ Группа пропущена")
                
        except (ValueError, KeyboardInterrupt):
            print("   ⚠️ Некорректный ввод или прерывание, пропускаем группу")
            continue
    
    if indices_to_remove:
        print(f"\n🗑️ Удаляем {len(indices_to_remove)} выбранных инструментов...")
        
        # Удаляем выбранные инструменты
        df_cleaned = df.drop(indices_to_remove).reset_index(drop=True)
        
        print(f"✅ Очистка завершена!")
        print(f"   Исходное количество: {len(df)}")
        print(f"   После очистки: {len(df_cleaned)}")
        print(f"   Удалено: {len(df) - len(df_cleaned)}")
        
        # Проверяем результат
        remaining_duplicates = df_cleaned.groupby(base_code_column).size()
        remaining_duplicates = remaining_duplicates[remaining_duplicates > 1]
        
        if len(remaining_duplicates) == 0:
            print("✅ Все дубликаты успешно удалены!")
        else:
            print(f"⚠️ Остались дубликаты: {len(remaining_duplicates)} групп")
        
        return df_cleaned
    else:
        print("ℹ️ Ничего не выбрано для удаления.")
        return df

# Попробуем использовать простую версию, если интерактивная не работает
try:
    if 'filtered_securities' in locals() and not filtered_securities.empty:
        print("🔧 Запускаем простой диалог удаления дубликатов...")
        filtered_securities_cleaned = remove_duplicates_simple(filtered_securities)
        
        # Обновляем переменную
        filtered_securities = filtered_securities_cleaned
        print(f"\n📊 Итоговое количество инструментов: {len(filtered_securities)}")
    else:
        print("❌ Нет данных для обработки дубликатов")
except Exception as e:
    print(f"❌ Ошибка при обработке дубликатов: {e}")
    print("Продолжаем с исходными данными...")


🔧 Запускаем простой диалог удаления дубликатов...
🔍 Найдено 1 групп дубликатов:

📋 Группа 'T' (2 инструментов):
  1. TBZ5 - T-12.25 (цена: 3222.0)
  2. TIZ5 - T-12.25 (цена: 3222.0)

❓ Какие инструменты из группы 'T' удалить?
   Введите номера через запятую (например: 1,3) или 'all' для удаления всех кроме первого:
   ✅ Будет удалено 1 инструментов

🗑️ Удаляем 1 выбранных инструментов...
✅ Очистка завершена!
   Исходное количество: 45
   После очистки: 44
   Удалено: 1
✅ Все дубликаты успешно удалены!

📊 Итоговое количество инструментов: 44


## 6. Создание основы для построения портфеля

Создаем основу для портфеля с расчетом весовых коэффициентов относительно индикаторного инструмента.


In [26]:
if SEC_IND_PRICE is not None and not filtered_securities.empty:
    print(f"Создаем основу портфеля с {NUM_PARAMS} параметрами для каждого инструмента")
    print(f"Индикаторный инструмент: {SEC_IND} (цена: {SEC_IND_PRICE:,.2f} руб.)")
    
    # Создаем список для хранения данных портфеля
    portfolio_basis = []
    
    for idx, row in filtered_securities.iterrows():
        sec_0 = row['SECID']
        sec_0_price = row['full_price']
        sec_0_GO = row['INITIALMARGIN']
        sec_1 = SEC_IND
        
        # Расчет весового коэффициента W_0
        W_0 = (sec_0_price / SEC_IND_PRICE * 1000000) // 1000 / 1000
        
        # Расчет float значений
        float_1 = max(1, int(sec_0_price / 100 * float_1_x))
        float_2 = max(1, int(sec_0_price / 100 * float_2_x))
        float_3 = float_3_x
        float_4 = float_4_x
        
        # Формирование колонки SM
        SM = f"{float_1};{float_2};{float_3};{float_4}"
        
        portfolio_basis.append({
            'Sec_0': sec_0,
            'sec_0_price': sec_0_price,
            'sec_0_GO': sec_0_GO,
            'sec_1': sec_1,
            'W_0': W_0,
            'SM': SM
        })
    
    # Преобразуем в DataFrame
    portfolio_basis_df = pd.DataFrame(portfolio_basis)
    
    print(f"\nСоздана основа портфеля для {len(portfolio_basis_df)} инструментов")
    print("\nПримеры записей:")
    # display(portfolio_basis_df.head())
    
    # Статистика по весовым коэффициентам
    print(f"\nСтатистика весовых коэффициентов W_0:")
    print(f"Минимум: {portfolio_basis_df['W_0'].min():.6f}")
    print(f"Максимум: {portfolio_basis_df['W_0'].max():.6f}")
    print(f"Среднее: {portfolio_basis_df['W_0'].mean():.6f}")
    print(f"Медиана: {portfolio_basis_df['W_0'].median():.6f}")
    
else:
    print("Ошибка: Не удалось создать основу портфеля.")
    if SEC_IND_PRICE is None:
        print("- Индикаторный инструмент не найден")
    if filtered_securities.empty:
        print("- Нет отфильтрованных инструментов")


Создаем основу портфеля с 1 параметрами для каждого инструмента
Индикаторный инструмент: MXZ5 (цена: 281,500.00 руб.)

Создана основа портфеля для 44 инструментов

Примеры записей:

Статистика весовых коэффициентов W_0:
Минимум: 0.002000
Максимум: 0.402000
Среднее: 0.054682
Медиана: 0.023000


## 7. Размножение строк по NUM_PARAMS

Создаем strategy_id для каждой комбинации инструмента и параметра, размножая строки по количеству параметров.


In [27]:
if 'portfolio_basis_df' in locals() and not portfolio_basis_df.empty:
    print("Размножаем строки по NUM_PARAMS...")
    
    # Создаем список параметров от 1 до NUM_PARAMS
    params_list = list(range(1, NUM_PARAMS + 1))
    print(f"Список параметров: {params_list}")
    
    # Создаем размноженные строки
    expanded_portfolio = []
    
    for idx, row in portfolio_basis_df.iterrows():
        for param_num in params_list:
            # Создаем strategy_id
            strategy_id = f"{row['Sec_0']}_{param_num}"
            
            # Копируем все данные строки
            expanded_row = row.copy()
            expanded_row['strategy_id'] = strategy_id
            expanded_row['param_num'] = param_num
            
            # Переупорядочиваем колонки, чтобы strategy_id была первой
            expanded_row = expanded_row.reindex(['strategy_id'] + [col for col in expanded_row.index if col != 'strategy_id'])
            
            expanded_portfolio.append(expanded_row)
    
    # Преобразуем в DataFrame
    expanded_portfolio_df = pd.DataFrame(expanded_portfolio)
    
    print(f"\nРазмножение завершено:")
    print(f"Исходных инструментов: {len(portfolio_basis_df)}")
    print(f"Параметров на инструмент: {NUM_PARAMS}")
    print(f"Итого строк: {len(expanded_portfolio_df)}")
    print(f"Ожидалось: {len(portfolio_basis_df) * NUM_PARAMS}")
    
    print("\nПримеры strategy_id:")
    sample_strategies = expanded_portfolio_df['strategy_id'].head(10).tolist()
    print(sample_strategies)
    
    print("\nПримеры записей:")
    display(expanded_portfolio_df[['strategy_id', 'Sec_0', 'param_num', 'W_0', 'SM']].head())
    
    # Проверяем уникальность strategy_id
    unique_strategies = expanded_portfolio_df['strategy_id'].nunique()
    total_strategies = len(expanded_portfolio_df)
    print(f"\nПроверка уникальности:")
    print(f"Уникальных strategy_id: {unique_strategies}")
    print(f"Общее количество строк: {total_strategies}")
    print(f"Совпадают: {unique_strategies == total_strategies}")
    
else:
    print("Ошибка: Не удалось размножить строки.")
    print("Убедитесь, что основа портфеля была создана успешно.")

expanded_portfolio_df = expanded_portfolio_df.reset_index(drop=True)


Размножаем строки по NUM_PARAMS...
Список параметров: [1]

Размножение завершено:
Исходных инструментов: 44
Параметров на инструмент: 1
Итого строк: 44
Ожидалось: 44

Примеры strategy_id:
['AFZ5_1', 'AKZ5_1', 'ALZ5_1', 'BNZ5_1', 'CMZ5_1', 'FLZ5_1', 'FSZ5_1', 'GKZ5_1', 'GZZ5_1', 'HDZ5_1']

Примеры записей:


,strategy_id,Sec_0,param_num,W_0,SM
0,AFZ5_1,AFZ5,1,0.020,1;1;0.01;0.05
1,AKZ5_1,AKZ5,1,0.052,1;2;0.01;0.05
2,ALZ5_1,ALZ5,1,0.016,1;1;0.01;0.05
3,BNZ5_1,BNZ5,1,0.005,1;1;0.01;0.05
4,CMZ5_1,CMZ5,1,0.026,1;1;0.01;0.05



Проверка уникальности:
Уникальных strategy_id: 44
Общее количество строк: 44
Совпадают: True


In [21]:
expanded_portfolio_df.head()

,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,param_num
0,AFZ5_1,AFZ5,5793.0,1538.73,MXZ5,0.020,1;1;0.01;0.05,1
1,AKZ5_1,AKZ5,14687.0,3831.65,MXZ5,0.052,4;2;0.01;0.05,1
2,ALZ5_1,ALZ5,4627.0,1235.04,MXZ5,0.016,1;1;0.01;0.05,1
3,BNZ5_1,BNZ5,1600.0,544.78,MXZ5,0.005,1;1;0.01;0.05,1
4,CMZ5_1,CMZ5,7539.0,2631.05,MXZ5,0.026,2;1;0.01;0.05,1


## 8. Формирование финального каркаса портфеля

Создаем финальную структуру портфеля с анализом результатов.


## 8.1. Тестирование новой функции build_portfolio_advanced

Теперь протестируем новую функцию `build_portfolio_advanced`, которая может работать с любым DataFrame и правильно распределять капитал по группам инструментов и параметрам.


In [28]:

# Импортируем новую функцию
from tvr_service.pipeline import build_portfolio_advanced

if 'expanded_portfolio_df' in locals() and not expanded_portfolio_df.empty:
    print("⚠️  expanded_portfolio_df не найден.")
    

    
    # Теперь применяем новую функцию
    portfolio_with_allocation = build_portfolio_advanced(
        data=expanded_portfolio_df,
        capital=CAPITAL,
        security_column="Sec_0",
        price_column="sec_0_GO",
        param_num_column="param_num",
        ensure_min_contract=True
    )
    
    print(f"Результат: {len(portfolio_with_allocation)} строк")
    print("\nПример результата:")
    display(portfolio_with_allocation[['strategy_id', 'Sec_0', 'param_num', 'sec_0_price', 'allocation', 'estimated_lots', 'used_capital', 'unused_capital']].head(10))
    
else:
    print("❌ Ошибка: Не найдены необходимые данные.")
    print("Доступные переменные:")
    available_vars = [var for var in locals() if 'portfolio' in var.lower() or 'expanded' in var.lower()]
    if available_vars:
        print(f"  - {', '.join(available_vars)}")
    else:
        print("  - Нет переменных с 'portfolio' или 'expanded' в названии")
    print("\nУбедитесь, что выполнили все предыдущие шаги блокнота.")

# Анализ результатов распределения капитала
if 'portfolio_with_allocation' in locals():
    print("=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ КАПИТАЛА ===\n")
    
    # Общая статистика
    total_used = portfolio_with_allocation['used_capital'].sum()
    total_unused = portfolio_with_allocation['unused_capital'].sum()
    utilization_rate = total_used / CAPITAL * 100
    
    print(f"Общий капитал: {CAPITAL:,.0f} руб.")
    print(f"Использовано: {total_used:,.0f} руб. ({utilization_rate:.1f}%)")
    print(f"Неиспользовано: {total_unused:,.0f} руб. ({100-utilization_rate:.1f}%)")
    
    # Статистика по инструментам
    print(f"\n=== СТАТИСТИКА ПО ИНСТРУМЕНТАМ ===")
    instrument_stats = portfolio_with_allocation.groupby('Sec_0').agg({
        'allocation': 'first',  # allocation одинаковая для всех строк инструмента
        'estimated_lots': 'sum',
        'used_capital': 'sum',
        'unused_capital': 'sum',
        'param_num': 'count'
    }).rename(columns={'param_num': 'param_count'})
    
    print(f"Уникальных инструментов: {len(instrument_stats)}")
    print(f"Капитал на инструмент: {CAPITAL / len(instrument_stats):,.0f} руб.")
    print("\nТоп-5 инструментов по используемому капиталу:")
    top_instruments = instrument_stats.nlargest(44, 'used_capital')[['used_capital', 'estimated_lots', 'param_count']]
    display(top_instruments)
    
    
    
else:
    print("Переменная portfolio_with_allocation не найдена.")


⚠️  expanded_portfolio_df не найден.
Результат: 44 строк

Пример результата:


,strategy_id,Sec_0,param_num,sec_0_price,allocation,estimated_lots,used_capital,unused_capital
0,AFZ5_1,AFZ5,1,5793.0,6818.181818,4,6154.92,663.261818
1,AKZ5_1,AKZ5,1,14687.0,6818.181818,1,3831.65,2986.531818
2,ALZ5_1,ALZ5,1,4627.0,6818.181818,5,6175.20,642.981818
3,BNZ5_1,BNZ5,1,1600.0,6818.181818,12,6537.36,280.821818
4,CMZ5_1,CMZ5,1,7539.0,6818.181818,2,5262.10,1556.081818
5,FLZ5_1,FLZ5,1,8367.0,6818.181818,4,6205.92,612.261818
6,FSZ5_1,FSZ5,1,6850.0,6818.181818,4,5783.40,1034.781818
7,GKZ5_1,GKZ5,1,1277.0,6818.181818,28,6753.60,64.581818
8,GZZ5_1,GZZ5,1,12441.0,6818.181818,2,4582.18,2236.001818
9,HDZ5_1,HDZ5,1,3543.0,6818.181818,5,6000.20,817.981818


=== АНАЛИЗ РАСПРЕДЕЛЕНИЯ КАПИТАЛА ===

Общий капитал: 300,000 руб.
Использовано: 306,525 руб. (102.2%)
Неиспользовано: 30,814 руб. (-2.2%)

=== СТАТИСТИКА ПО ИНСТРУМЕНТАМ ===
Уникальных инструментов: 44
Капитал на инструмент: 6,818 руб.

Топ-5 инструментов по используемому капиталу:


,used_capital,estimated_lots,param_count
Sec_0,,,
NKZ5,25195.18,1,1
LKZ5,13819.97,1,1
TTZ5,12287.84,1,1
RNZ5,9554.61,1,1
SGZ5,8971.47,1,1
IRZ5,8419.58,1,1
WUZ5,6811.56,18,1
RTZ5,6783.05,5,1
TBZ5,6769.07,11,1


In [14]:
portfolio_with_allocation


,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,param_num,allocation,estimated_lots,used_capital,unused_capital
0,AFZ5_1,AFZ5,5793.0,1538.73,MXZ5,0.020,1;1;0.01;0.05,1,3077.460000,2,3077.46,0.000000
1,AFZ5_2,AFZ5,5793.0,1538.73,MXZ5,0.020,1;1;0.01;0.05,2,2201.991818,1,1538.73,663.261818
2,AFZ5_3,AFZ5,5793.0,1538.73,MXZ5,0.020,1;1;0.01;0.05,3,1538.730000,1,1538.73,0.000000
3,AKZ5_1,AKZ5,14687.0,3831.65,MXZ5,0.052,4;1;0.01;0.05,1,3831.650000,1,3831.65,0.000000
4,AKZ5_2,AKZ5,14687.0,3831.65,MXZ5,0.052,4;1;0.01;0.05,2,2986.531818,0,0.00,2986.531818
...,...,...,...,...,...,...,...,...,...,...,...,...
127,X5Z5_2,X5Z5,2815.0,535.71,MXZ5,0.010,1;1;0.01;0.05,2,2142.840000,4,2142.84,0.000000
128,X5Z5_3,X5Z5,2815.0,535.71,MXZ5,0.010,1;1;0.01;0.05,3,2142.840000,4,2142.84,0.000000
129,YDZ5_1,YDZ5,4192.0,774.70,MXZ5,0.014,1;1;0.01;0.05,1,2324.100000,3,2324.10,0.000000
130,YDZ5_2,YDZ5,4192.0,774.70,MXZ5,0.014,1;1;0.01;0.05,2,2324.100000,3,2324.10,0.000000


In [29]:
## 9. Формирование финальной таблицы портфеля

# Создаем финальную таблицу со всеми исходными колонками и добавленным столбцом V_0
if 'portfolio_with_allocation' in locals():
    print("Создаем финальную таблицу портфеля...")
    
    # Копируем все данные из исходной таблицы
    Final_portfolio_all = expanded_portfolio_df.copy()
    
    # Добавляем столбец V_0 с данными из estimated_lots
    Final_portfolio_all['V_0'] = portfolio_with_allocation['estimated_lots']
    
       
else:
    print("❌ Ошибка: portfolio_with_allocation не найден.")
    print("Убедитесь, что предыдущие ячейки выполнены успешно.")


Создаем финальную таблицу портфеля...


In [40]:
Final_portfolio_all.to_csv('porfolio_base.csv', sep = ',')

In [43]:
from pandas import read_csv


port_base = read_csv('porfolio_base.csv', header=0)

In [44]:
port_base

,Unnamed: 0,strategy_id,Sec_0,sec_0_price,sec_0_GO,sec_1,W_0,SM,param_num,V_0
0,0,AFZ5_1,AFZ5,5793.0,1538.73,MXZ5,0.020,1;1;0.01;0.05,1,4
1,1,AKZ5_1,AKZ5,14687.0,3831.65,MXZ5,0.052,1;2;0.01;0.05,1,1
2,2,ALZ5_1,ALZ5,4627.0,1235.04,MXZ5,0.016,1;1;0.01;0.05,1,5
3,3,BNZ5_1,BNZ5,1600.0,544.78,MXZ5,0.005,1;1;0.01;0.05,1,12
4,4,CMZ5_1,CMZ5,7539.0,2631.05,MXZ5,0.026,1;1;0.01;0.05,1,2
5,5,FLZ5_1,FLZ5,8367.0,1551.48,MXZ5,0.029,1;1;0.01;0.05,1,4
6,6,FSZ5_1,FSZ5,6850.0,1445.85,MXZ5,0.024,1;1;0.01;0.05,1,4
7,7,GKZ5_1,GKZ5,1277.0,241.20,MXZ5,0.004,1;1;0.01;0.05,1,28
8,8,GZZ5_1,GZZ5,12441.0,2291.09,MXZ5,0.044,1;2;0.01;0.05,1,2
9,9,HDZ5_1,HDZ5,3543.0,1200.04,MXZ5,0.012,1;1;0.01;0.05,1,5
